In [2]:
import os
os.getcwd()


'E:\\StudyTime\\Fall\\Data Viz\\diabetes-viz\\coding'

In [3]:
import pandas as pd
import altair as alt
alt.data_transformers.enable('default', max_rows=None) 

DataTransformerRegistry.enable('default')

In [4]:
import pandas as pd

df = pd.read_csv('../data/diabetes_01.csv')

# check
df.head()


,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0,1,1,1,40,1,0,0,0,0,...,1,0,5,18,15,1,0,9,4,3
1,0,0,0,0,25,1,0,0,1,0,...,0,1,3,0,0,0,0,7,6,1
2,0,1,1,1,28,0,0,0,0,1,...,1,1,5,30,30,1,0,9,4,8
3,0,1,0,1,27,0,0,0,1,1,...,1,0,2,0,0,0,0,11,3,6
4,0,1,1,1,24,0,0,0,1,1,...,1,0,2,3,0,0,0,11,5,4


In [5]:
df['Diabetes_012'] = df['Diabetes_012'].map({
    0.0: 'No Diabetes',
    1.0: 'Prediabetes',
    2.0: 'Diabetes'
})

df['HighBP'] = df['HighBP'].map({
    0.0: 'No High BP',
    1.0: 'High BP'
})

df['Sex'] = df['Sex'].map({
    0.0: 'Female',
    1.0: 'Male'
})

df.head()

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,No Diabetes,High BP,1,1,40,1,0,0,0,0,...,1,0,5,18,15,1,Female,9,4,3
1,No Diabetes,No High BP,0,0,25,1,0,0,1,0,...,0,1,3,0,0,0,Female,7,6,1
2,No Diabetes,High BP,1,1,28,0,0,0,0,1,...,1,1,5,30,30,1,Female,9,4,8
3,No Diabetes,High BP,0,1,27,0,0,0,1,1,...,1,0,2,0,0,0,Female,11,3,6
4,No Diabetes,High BP,1,1,24,0,0,0,1,1,...,1,0,2,3,0,0,Female,11,5,4


In [ ]:
chart = alt.Chart(df).mark_bar().encode(
    x=alt.X('Diabetes_012:N', title='Diabetes Status'),
    y=alt.Y('count()', stack='normalize', title='Percentage'),
    color=alt.Color('HighBP:N', title='High Blood Pressure'),
    tooltip=['Diabetes_012', 'HighBP', 'count()']
).properties(
    title='Share of High Blood Pressure within Each Diabetes Group'
)

chart

In [ ]:
alt.Chart(df).mark_arc(innerRadius=60).encode(
    theta='count()',
    color=alt.Color('Diabetes_012:N', title='Diabetes Status'),
    tooltip=['Diabetes_012', 'count()']
)


In [ ]:
order = ['Diabetes', 'Prediabetes', 'No Diabetes']


chart = (
    alt.Chart(df)
    .mark_bar()
    .encode(
        y=alt.Y('Diabetes_012:N', sort=order, title='Diabetes Status'),
        x=alt.X('count()', stack='normalize', title='Percentage'),
        color=alt.Color(
            'HighBP:N',
            title='High Blood Pressure',
            scale=alt.Scale(
                domain=['No High BP', 'High BP'],
                range=['#94b4e2', '#e15759']  
            )
        ),
        tooltip=[
            'Diabetes_012',
            'HighBP',
            alt.Tooltip('count()', title='Number of People')
        ]
    )
    .properties(
        title='People with Diabetes Are More Likely to Have High Blood Pressure',
        width=500,
        height=250
    )
)

chart 

In [ ]:
chart = (
    alt.Chart(df)
    .mark_arc()
    .encode(
        theta='count()',
        color=alt.Color(
            'Sex:N',
            scale=alt.Scale(
                domain=['Female', 'Male'],
                range=['#F4B6C2', '#A7C7E7']
            )
        )
    )
    .transform_filter("datum.Diabetes_012 == 'Diabetes'")
    .properties(
        title='Gender Distribution among People with Diabetes'
    )
)

chart


In [ ]:
chart = (
    alt.Chart(df)
    .transform_filter("datum.Diabetes_012 == 'Diabetes'")
    .mark_arc(innerRadius=60)
    .encode(
        theta='count()',
        color=alt.Color('Sex:N', title='Gender Distribution'),
        tooltip=['Sex', 'count()']
    )
    .properties(
        title='Gender Distribution Among People with Diabetes'
    )
)

chart


In [ ]:
def categorize_bmi(bmi):
    if bmi < 18.5:
        return 'Underweight'
    elif 18.5 <= bmi < 25:
        return 'Normal'
    elif 25 <= bmi < 30:
        return 'Overweight'
    else:
        return 'Obese'

df['BMI_Category'] = df['BMI'].apply(categorize_bmi)

df_diabetes = df[df['Diabetes_012'] == 'Diabetes']

agg_df = (
    df_diabetes['BMI_Category']
    .value_counts()
    .rename_axis('BMI_Category')
    .reset_index(name='Count')
)

chart = (
    alt.Chart(agg_df)
    .mark_bar(size=50)
    .encode(
        x=alt.X(
            'BMI_Category:N',
            sort=['Underweight', 'Normal', 'Overweight', 'Obese'],
            title='BMI Category'
        ),
        y=alt.Y(
            'Count:Q',
            title='Number of Diabetic Individuals'
        ),
        color=alt.Color(
            'BMI_Category:N',
            title='BMI Category',
            scale=alt.Scale(
                domain=['Underweight', 'Normal', 'Overweight', 'Obese'],
                range=['#A7C7E7', '#8FD694', '#F4C27F', '#E15759']  
            )
        ),
        tooltip=['BMI_Category', 'Count']
    )
    .properties(
        title='Number of Diabetic Individuals by BMI Category',
        width=500,
        height=300
    )
)

chart


In [ ]:
df_long = df.melt(
    id_vars=['Diabetes_012'],                                   #CI asked ChatGPT to explain how to reshape my dataset 
                                                                #so I could compare general, mental, and physical
                                                                #  health measures by diabetes status in one visualization. It suggested 'melt'
    value_vars=['GenHlth', 'MentHlth', 'PhysHlth'],
    var_name='HealthType',
    value_name='Value'
)


label_map = {
    'GenHlth': 'General Health',
    'MentHlth': 'Mental Health',
    'PhysHlth': 'Physical Health'
}
df_long['HealthType'] = df_long['HealthType'].map(label_map)

# Set x-axis order
order = ['No Diabetes', 'Prediabetes', 'Diabetes']

# Create chart
chart = (
    alt.Chart(df_long)
    .mark_bar()
    .encode(
        x=alt.X('Diabetes_012:O', sort=order),
        y=alt.Y('mean(Value):Q', title='Number of Days Sick (Average)'),
        color=alt.Color(
            'HealthType:N',
            scale=alt.Scale(
                domain=['General Health', 'Mental Health', 'Physical Health'],
                range=['#A7C957','#70C1B3','#FF7E6B']  
            ),
            title='Health Measure'
        ),
        column=alt.Column(
            'HealthType:N',
            title=None,
            header=alt.Header(labelAngle=0)    #ChatGPT prompt:how do i keep header label horizontal?
                                                # labelAngle = 0 → text is horizontal.
        )
    )
    .properties(
        title='Average Health Measures by Diabetes Status'
    )
)

chart


In [ ]:

df['Smoker'] = df['Smoker'].map({0: 'Non-Smokers', 1: 'Smokers'})

# Pie chart
chart = (
    alt.Chart(df)
    .mark_arc()
    .encode(
        theta='count()',
        color=alt.Color(
            'Smoker:N',
            title='Smoking Status',
            scale=alt.Scale(
                domain=['Non-Smokers', 'Smokers'],
                range=['#A9A9A9', '#8B4513']  
            )
        ),
        tooltip=[
            alt.Tooltip('Smoker:N', title='Smoking Status'),
            alt.Tooltip('count(Smoker):Q', title='Count')
        ]
    )
    .properties(title='Distribution of Smokers vs Non-Smokers')
)

chart




In [ ]:
chart2 = (
    alt.Chart(df)
    .transform_filter("datum.Diabetes_012 == 'Diabetes'")
    .mark_rect()
    .encode(
        x=alt.X('Education:O', title='Education Level'),
        y=alt.Y('Income:O', title='Income Level'),
        color=alt.Color('count()', title='Number of Diabetic Individuals'),
        tooltip=['Education', 'Income', 'count()']
    )
    .properties(
        title='Diabetes Distribution by Education and Income Levels',
        width=500,
        height=350
    )
)
chart2


In [ ]:

agg = df.groupby(['Diabetes_012', 'DiffWalk']).size().reset_index(name='Count')    #ChatGPTprompt: how do i create a bubble graph for three datasets categories, dont provide code, provide study guide
                                                                                   #use groupby function     

agg['DiffWalk'] = agg['DiffWalk'].map({0: 'No Difficulty', 1: 'Difficulty Walking'})

order = ['No Diabetes', 'Prediabetes', 'Diabetes']

chart = (
    alt.Chart(agg)
    .mark_circle(opacity=0.8)
    .encode(
        x=alt.X('Diabetes_012:N',                                                   
                sort=order,
                title='Diabetes Status',
                axis=alt.Axis(labelAngle=0)),
        y=alt.Y('DiffWalk:N',
                title=None,
                axis=alt.Axis(labelAngle=0, labelPadding=5)),
        size=alt.Size('Count:Q',
                      title='Number of People',
                      scale=alt.Scale(range=[100, 2000]),  
                      legend=None),
        color=alt.Color('DiffWalk:N',
                        title='Walking Ability',
                        scale=alt.Scale(
                            domain=['No Difficulty', 'Difficulty Walking'],
                            range=['#66BB6A', '#EF5350']
                        )),
        tooltip=['Diabetes_012', 'DiffWalk', 'Count']
    )
    .properties(
        title='Walking Difficulty by Diabetes Status',
        width=500,
        height=180,  
        padding=10
    )
    .configure_axis(                                                                #I asked Chatgpt how to make the chart more visually appealing
                                                                                    #i.e. remove white spaces and it suggested configure 
        labelFontSize=12,
        titleFontSize=13
    )
    .configure_title(
        fontSize=16,
        anchor='middle'
    )
)

chart
